In [1]:
from sklearn.metrics.pairwise import pairwise_distances
from tensorflow.keras.utils import multi_gpu_model
from model import identity_block, conv_block
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import layers
import numpy as np
import h5py
from os.path import isfile, join
import os
import matplotlib.pyplot as plt

In [2]:
lr=1e-4
epochs=250
decay=1e-2

data_path="data/CSI_48_mc"
data_files = [join(data_path, f) for f in os.listdir(data_path) if isfile(join(data_path, f)) and f.endswith(".h5")]
history = []

hf = h5py.File(data_files[0], 'r')
train_classes = np.array(hf.get('labels')).astype(str)
num_classes = train_classes.shape[0]
X_train = np.expand_dims(hf.get('X_train'), axis=-1)
X_test = np.expand_dims(hf.get('X_test'), axis=-1)
y_train = np.eye(num_classes)[hf.get('y_train')]
y_test =np.eye(num_classes)[hf.get('y_test')]
hf.close()
    
input_layer = layers.Input(shape=(X_train.shape[1:]))

x = layers.Conv2D(64, (7, 7), strides=(2, 2))(input_layer)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, [64, 256], "relu")
x = identity_block(x, [64, 256], "relu")

x = conv_block(x, [128, 512], "relu")
x = identity_block(x, [128, 512], "relu")

x = conv_block(x, [256, 1024], "relu")
x = identity_block(x, [256, 1024], "relu")

x = conv_block(x, [512, 2048], "relu")
x = identity_block(x, [512, 2048], "relu")

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(num_classes, activation=None)(x)
x = layers.Activation('softmax')(x)

model_base = Model(inputs=input_layer, outputs=x)
model_base.summary()
model = multi_gpu_model(model_base, gpus=4)

for data_ind in range(len(data_files)):
    print("MC:{}".format(data_ind))

    hf = h5py.File(data_files[data_ind], 'r')
    train_classes = np.array(hf.get('labels')).astype(str)
    num_classes = train_classes.shape[0]
    X_train = np.expand_dims(hf.get('X_train'), axis=-1)
    X_test = np.expand_dims(hf.get('X_test'), axis=-1)
    y_train = np.eye(num_classes)[hf.get('y_train')]
    y_test =np.eye(num_classes)[hf.get('y_test')]
    hf.close()

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr, decay=decay), metrics=['acc'])
    history.append(model.fit(x=X_train, y=y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=2))

MC:0
(1631, 2000, 270, 1) (288, 2000, 270, 1) (1631, 49) (288, 49)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2000, 270, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 997, 132, 64) 3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 997, 132, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 997, 132, 64) 0           batch_normalization[0][0]        
__________________________________________

ResourceExhaustedError: OOM when allocating tensor with shape[32,64,997,132] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_63/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 2, 2], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_63/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_63/Conv2D/ReadVariableOp)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
data_dir="data/CSI_intruder_pca.h5"

hf = h5py.File(data_dir, 'r')
classes = np.array(hf.get('labels')).astype(str)
X_data = np.expand_dims(hf.get('X_data'), axis=-1)
y_data = np.eye(classes.shape[0])[hf.get('y_data')]
hf.close()

print(X_data.shape)

In [ ]:
for i in range(X_data.shape[0]):
    print("{:16}{}".format(train_classes[np.argmax(model.predict(X_data), axis=-1)[i]], classes[np.argmax(y_data, axis=-1)[i]]))